In [1]:
# Install required packages
!pip install torch
!pip install mixture_of_experts
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from datasets import load_dataset
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import numpy as np
import os
import urllib.request
from torch.optim import Adam
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from mixture_of_experts import MoE

# Define your Mixture-of-Experts model
class MyMoEModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_experts):
        super(MyMoEModel, self).__init__()
        
        # LSTM layers
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        
        # Mixture-of-Experts layer
        self.moe = MoE(
            dim=hidden_dim,
            num_experts=num_experts,
            hidden_dim=hidden_dim * 4,
            activation=nn.LeakyReLU,
            second_policy_train='random',
            second_policy_eval='random',
            second_threshold_train=0.2,
            second_threshold_eval=0.2,
            capacity_factor_train=1.25,
            capacity_factor_eval=2.,
            loss_coef=1e-2
        )
        
        # Output layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out, aux_loss = self.moe(out)
        output = self.fc(out)
        return output, aux_loss

# Example usage
input_dim = 512
hidden_dim = 256
output_dim = 10
num_experts = 16

model = MyMoEModel(input_dim, hidden_dim, output_dim, num_experts)
inputs = torch.randn(4, 1024, input_dim)
outputs, auxiliary_loss = model(inputs)
print("Output shape:", outputs.shape)
print("Auxiliary loss:", auxiliary_loss.item())


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Output shape: torch.Size([4, 1024, 10])
Auxiliary loss: 0.014753254130482674


In [7]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForTokenClassification
from datasets import load_dataset

# Load CoNLL-2003 dataset
dataset = load_dataset("conll2003")

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# Corrected line: Get the number of unique labels
import numpy as np

# Flatten the list of ner_tags and find the number of unique labels
num_labels = len(np.unique(np.concatenate(dataset["train"]["ner_tags"])))
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=num_labels)


# Prepare input data
def prepare_input_data(example):
    # Join the tokens into a single string for each sentence
    sentences = [' '.join(tokens) for tokens in example["tokens"]]
    # Tokenize each sentence
    input_data = [tokenizer(sentence, truncation=True, padding='max_length') for sentence in sentences]
    # Prepare the labels: if a sentence is shorter than the max length, pad it with -100s
    labels = [label + [-100]*(tokenizer.model_max_length - len(label)) for label in example["ner_tags"]]
    # Extract input_ids and attention_mask from input_data
    input_ids = [data['input_ids'] for data in input_data]
    attention_mask = [data['attention_mask'] for data in input_data]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}




train_data = dataset["train"].map(prepare_input_data, batched=True)
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Fine-tune the model
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in train_data:
        optimizer.zero_grad()
        outputs = model(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["labels"])
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_data)
    print(f"Epoch {epoch+1} - Average training loss: {average_loss:.4f}")

# Evaluate the model
eval_data = dataset["validation"].map(prepare_input_data, batched=True)
eval_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

model.eval()
total_loss = 0
with torch.no_grad():
    for batch in eval_data:
        outputs = model(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["labels"])
        total_loss += outputs.loss.item()

average_loss = total_loss / len(eval_data)
print(f"Average validation loss: {average_loss:.4f}")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

ValueError: not enough values to unpack (expected 2, got 1)

: 